<a href="https://colab.research.google.com/github/GGMmattos/PIC/blob/main/Experimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação das bibliotecas 

In [ ]:
#Importing needed libraries 
!pip install -U scikit-learn

from pandas.core.window.ewm import OnlineExponentialMovingWindow
import pandas as pd #Biblioteca para carregamento de arquivo csv
import numpy as np
import seaborn as sns #Para visualização de gráficos
import matplotlib.pyplot as plt #Para visualização de gráficos
import plotly.express as px #Biblioteca para geração de gráfico dinámico
%matplotlib inline
from collections import Counter


#NLTK tools for text processing 

import re, nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


#Modeling packages 

from sklearn.model_selection import train_test_split
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'


!pip install -U spacy
!python -m spacy download pt
import spacy



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Experimento com a HateBR



Sem pré-processamento

In [ ]:
from pandas.core.groupby import base
base_hatebr = pd.read_csv('/content/drive/MyDrive/PIC /Dataset/HateBR.csv')

In [4]:
base_hatebr.offensive_language.value_counts()

1    3500
0    3500
Name: offensive_language, dtype: int64

Dividindo em base de teste e base de treinamento



In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    base_hatebr.instagram_comments,
    base_hatebr.offensive_language, 
    test_size = 0.2, #20% das amostras vão para o conjunto de dados de teste
    random_state = 2022,
    stratify = base_hatebr.offensive_language
)

In [6]:
print('Shape of X_train', X_train.shape)
print('Shape of X_test', X_test.shape )

Shape of X_train (5600,)
Shape of X_test (1400,)


In [7]:
y_train.value_counts()

1    2800
0    2800
Name: offensive_language, dtype: int64

In [8]:
y_test.value_counts()

1    700
0    700
Name: offensive_language, dtype: int64

Treinamento com Naive Bayes




In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

#Criação do objeto Pipeline
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()), #using the ngram_range parameter
    ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)

#Obtem a precisão de X_test e e armazena em y_pred
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

              precision    recall  f1-score   support

           0       0.85      0.81      0.83       700
           1       0.82      0.86      0.84       700

    accuracy                           0.84      1400
   macro avg       0.84      0.84      0.84      1400
weighted avg       0.84      0.84      0.84      1400



Support Vector Machine(SVM)

In [10]:
from sklearn.svm import SVC

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('SVM', SVC( kernel = 'linear'))
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('SVM', SVC(kernel='linear'))])

              precision    recall  f1-score   support

           0       0.84      0.85      0.84       700
           1       0.85      0.84      0.84       700

    accuracy                           0.84      1400
   macro avg       0.84      0.84      0.84      1400
weighted avg       0.84      0.84      0.84      1400



Multilayer Perceptron(MLP) 

In [11]:
from sklearn.neural_network import MLPClassifier

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('MLP', MLPClassifier())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('MLP', MLPClassifier())])

              precision    recall  f1-score   support

           0       0.83      0.85      0.84       700
           1       0.85      0.83      0.84       700

    accuracy                           0.84      1400
   macro avg       0.84      0.84      0.84      1400
weighted avg       0.84      0.84      0.84      1400



 Regressão Logística(LR)

In [12]:
from sklearn.linear_model import LogisticRegression

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('LR', LogisticRegression())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('LR', LogisticRegression())])

              precision    recall  f1-score   support

           0       0.83      0.82      0.83       700
           1       0.82      0.83      0.83       700

    accuracy                           0.83      1400
   macro avg       0.83      0.83      0.83      1400
weighted avg       0.83      0.83      0.83      1400



Com pre-processamento



In [13]:
# comments = [] 
# stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

# for words in base_hatebr ['instagram_comments']:
#   only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
#   tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
#   lower_case = [l.lower() for l in tokens] # converter all letters to low case 
#   filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
#   comments.append(' '.join(filtered_result))
  
  # for i in range (len(comments)):
  #   words = nltk.word_tokenize(comments[i])
  #   newwords = [lemmatizer.lemmatize(word) for word in words]
  #   comments[i] = ' '.join(newwords)

Função de pré-processamento 



*   Foi notado que ao realizar o procedimento de deixar somente letras o modelo teve piora de pontuação F1, sem a função o texto continua com emojis e caracteres especiais...



In [14]:
npl = spacy.load("pt_core_news_sm")
def preprocess(text):
  comentarios = text.lower()
  # only_latters =re.sub("[^a-zA-Z]", " ", comentarios) # somente letras 
  doc = npl(comentarios)
  filtered_result = []
  for token in doc:
    if token.is_stop or token.is_punct :
      continue
    filtered_result.append(token.lemma_)
  return " ".join(filtered_result)

Criando uma nova coluna com o texto com pré-processamento

In [15]:
# print(preprocess(base_hatebr['instagram_comments']))
base_hatebr['preprocessed_comments'] =  base_hatebr['instagram_comments'].apply(preprocess)

In [16]:
base_hatebr

,instagram_comments,offensive_language,offensiveness_levels,hate_speech,preprocessed_comments
0,este lixo ...,1,1,-1,lixo
1,Mais um lixo,1,1,-1,lixo
2,Essa nao tem vergonha na cara!!,1,2,-1,nao vergonha cara
3,Essa mulher é doente.pilantra!,1,3,-1,mulher doente.pilantra
4,Comunista safada...,1,2,"5,8",comunista safar
...,...,...,...,...,...
6995,Time perdendo a credibilidade,0,0,0,time perder credibilidade
6996,Siga em frente Presidente Bolsonaro! Afinal o ...,0,0,0,siga frente presidente bolsonaro afinal intere...
6997,Tantas coisas importantes para resolver e fica...,0,0,0,tanto coisa importante resolver ficar dar ênfa...
6998,"A TIME escolhe quem eles quiser, isso não sign...",0,0,0,time escolhe querer significar


In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    base_hatebr.	preprocessed_comments,
    base_hatebr.offensive_language, 
    test_size = 0.2, #20% das amostras vão para o conjunto de dados de teste
    random_state = 2022,
    stratify = base_hatebr.offensive_language
)

In [18]:
print('Shape of X_train', X_train.shape)
print('Shape of X_test', X_test.shape )

Shape of X_train (5600,)
Shape of X_test (1400,)


Treinamento com Naive Bayes

In [19]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report


#Criação do objeto Pipeline
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()), #using the ngram_range parameter
    ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)

#Obtem a precisão de X_test e e armazena em y_pred
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

              precision    recall  f1-score   support

           0       0.86      0.84      0.85       700
           1       0.84      0.86      0.85       700

    accuracy                           0.85      1400
   macro avg       0.85      0.85      0.85      1400
weighted avg       0.85      0.85      0.85      1400



In [20]:

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report #comparação entre os resultados 
accuracy_score(y_test, y_pred) #retorna a porcentagem dos acertos 
     

0.8492857142857143

Support Vector Machine(SVM)

In [21]:
from sklearn.svm import SVC

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('SVM', SVC(C=2.0, kernel = 'linear'))
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('SVM', SVC(C=2.0, kernel='linear'))])

              precision    recall  f1-score   support

           0       0.84      0.88      0.86       700
           1       0.87      0.83      0.85       700

    accuracy                           0.85      1400
   macro avg       0.85      0.85      0.85      1400
weighted avg       0.85      0.85      0.85      1400



Multilayer Perceptron(MLP)

In [22]:
from sklearn.neural_network import MLPClassifier

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('MLP', MLPClassifier())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('MLP', MLPClassifier())])

              precision    recall  f1-score   support

           0       0.82      0.88      0.85       700
           1       0.87      0.80      0.84       700

    accuracy                           0.84      1400
   macro avg       0.85      0.84      0.84      1400
weighted avg       0.85      0.84      0.84      1400



 Regressão Logística(LR)

In [23]:
from sklearn.linear_model import LogisticRegression

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('LR', LogisticRegression())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('LR', LogisticRegression())])

              precision    recall  f1-score   support

           0       0.84      0.87      0.85       700
           1       0.87      0.83      0.85       700

    accuracy                           0.85      1400
   macro avg       0.85      0.85      0.85      1400
weighted avg       0.85      0.85      0.85      1400



TOLD


# Experimento com a TOLD BR

Sem pré-processamento

In [24]:
told_br = pd.read_csv("/content/drive/MyDrive/PIC /Dataset/ToLD-BR_alpha.csv")


Tranformação em base binária

In [25]:
told_br["toxic"] = 0
told_br["toxic_1"] = 0
told_br["toxic_2"] = 0
told_br["toxic_3"] = 0
told_br["homophobia"] = 0
told_br["obscene"] = 0
told_br["insult"] = 0
told_br["racism"] = 0
told_br["misogyny"] = 0
told_br["xenophobia"] = 0
for i, row in told_br.iterrows():
    count = 0
    if 1 in list(row[["homophobia_1", "obscene_1", "insult_1", "racism_1", "misogyny_1", "xenophobia_1"]]):
        told_br.loc[i, "toxic_1"] = 1
        count += 1
    if 1 in list(row[["homophobia_2", "obscene_2", "insult_2", "racism_2", "misogyny_2", "xenophobia_2"]]):
        told_br.loc[i, "toxic_2"] = 1
        count += 1
    if 1 in list(row[["homophobia_3", "obscene_3", "insult_3", "racism_3", "misogyny_3", "xenophobia_3"]]):
        told_br.loc[i, "toxic_3"] = 1
        count += 1

    told_br.loc[i, "homophobia"] = np.sum(list(row[["homophobia_1", "homophobia_2", "homophobia_3"]]))
    
    told_br.loc[i, "obscene"] = np.sum(list(row[["obscene_1", "obscene_2", "obscene_3"]]))
    
    told_br.loc[i, "insult"] = np.sum(list(row[["insult_1", "insult_2", "insult_3"]]))
    
    told_br.loc[i, "racism"] = np.sum(list(row[["racism_1", "racism_2", "racism_3"]]))
        
    told_br.loc[i, "misogyny"] = np.sum(list(row[["misogyny_1", "misogyny_2", "misogyny_3"]]))

    told_br.loc[i, "xenophobia"] = np.sum(list(row[["xenophobia_1", "xenophobia_2", "xenophobia_3"]]))
        
    if count >= 1:
        told_br.loc[i, "toxic"] = 1

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    told_br.text,
    told_br.toxic, 
    test_size = 0.2, #20% das amostras vão para o conjunto de dados de teste
    random_state = 2022,
    stratify = told_br.toxic
)

In [27]:
print('Shape of X_train', X_train.shape)
print('Shape of X_test', X_test.shape )

Shape of X_train (16800,)
Shape of X_test (4200,)


Treinamento com Naive Bayes


In [28]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

#Criação do objeto Pipeline
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()), #using the ngram_range parameter
    ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)

#Obtem a precisão de X_test e e armazena em y_pred
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

              precision    recall  f1-score   support

           0       0.68      0.87      0.76      2349
           1       0.74      0.47      0.58      1851

    accuracy                           0.69      4200
   macro avg       0.71      0.67      0.67      4200
weighted avg       0.70      0.69      0.68      4200



Support Vector Machine(SVM)

In [29]:
from sklearn.svm import SVC

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('SVM', SVC( kernel = 'linear'))
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('SVM', SVC(kernel='linear'))])

              precision    recall  f1-score   support

           0       0.77      0.80      0.78      2349
           1       0.73      0.69      0.71      1851

    accuracy                           0.75      4200
   macro avg       0.75      0.75      0.75      4200
weighted avg       0.75      0.75      0.75      4200



Multilayer Perceptron(MLP) 

In [30]:
from sklearn.neural_network import MLPClassifier

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('MLP', MLPClassifier())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('MLP', MLPClassifier())])

              precision    recall  f1-score   support

           0       0.71      0.75      0.73      2349
           1       0.66      0.61      0.64      1851

    accuracy                           0.69      4200
   macro avg       0.69      0.68      0.68      4200
weighted avg       0.69      0.69      0.69      4200



 Regressão Logística(LR)

In [31]:
from sklearn.linear_model import LogisticRegression

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('LR', LogisticRegression())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('LR', LogisticRegression())])

              precision    recall  f1-score   support

           0       0.75      0.83      0.79      2349
           1       0.75      0.65      0.69      1851

    accuracy                           0.75      4200
   macro avg       0.75      0.74      0.74      4200
weighted avg       0.75      0.75      0.75      4200



In [32]:
#Comando para baixar o Dataset
told_br.to_csv("told_br_binary.csv")

Com pré-processamento

In [33]:
#ja foi criada acima, excluir após testes
npl = spacy.load("pt_core_news_sm")
def preprocess(text):
  comentarios = text.lower()
  # only_latters =re.sub("[^a-zA-Z]", " ", comentarios) # somente letras 
  doc = npl(comentarios)
  filtered_result = []
  for token in doc:
    if token.is_stop or token.is_punct :
      continue
    filtered_result.append(token.lemma_)
  return " ".join(filtered_result)

In [34]:
# print(preprocess(base_hatebr['instagram_comments']))
told_br['preprocessed_text'] =  told_br['text'].apply(preprocess)

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    told_br.preprocessed_text,
    told_br.toxic, 
    test_size = 0.2, #20% das amostras vão para o conjunto de dados de teste
    random_state = 2022,
    stratify = told_br.toxic
)

Treinamento com Naive Bayes


In [36]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()), 
    ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

              precision    recall  f1-score   support

           0       0.70      0.84      0.76      2349
           1       0.73      0.53      0.61      1851

    accuracy                           0.71      4200
   macro avg       0.71      0.69      0.69      4200
weighted avg       0.71      0.71      0.70      4200



Support Vector Machine(SVM)

In [37]:
from sklearn.svm import SVC

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('SVM', SVC( kernel = 'linear'))
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('SVM', SVC(kernel='linear'))])

              precision    recall  f1-score   support

           0       0.75      0.80      0.77      2349
           1       0.72      0.67      0.69      1851

    accuracy                           0.74      4200
   macro avg       0.74      0.73      0.73      4200
weighted avg       0.74      0.74      0.74      4200



Multilayer Perceptron(MLP) 

In [38]:
from sklearn.neural_network import MLPClassifier

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('MLP', MLPClassifier())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('MLP', MLPClassifier())])

              precision    recall  f1-score   support

           0       0.72      0.74      0.73      2349
           1       0.66      0.64      0.65      1851

    accuracy                           0.69      4200
   macro avg       0.69      0.69      0.69      4200
weighted avg       0.69      0.69      0.69      4200



 Regressão Logística(LR)

In [39]:
from sklearn.linear_model import LogisticRegression

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('LR', LogisticRegression())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('LR', LogisticRegression())])

              precision    recall  f1-score   support

           0       0.74      0.84      0.79      2349
           1       0.75      0.63      0.68      1851

    accuracy                           0.74      4200
   macro avg       0.75      0.73      0.74      4200
weighted avg       0.75      0.74      0.74      4200

